# 第二章、向量数据库的介绍及使用



## 一、向量数据库简介

向量数据库是用于高效计算和管理大量向量数据的解决方案。向量数据库是一种专门用于存储和检索向量数据（embedding）的数据库系统。它与传统的基于关系模型的数据库不同，它主要关注的是向量数据的特性和相似性。

在向量数据库中，数据被表示为向量形式，每个向量代表一个数据项。这些向量可以是数字、文本、图像或其他类型的数据。向量数据库使用高效的索引和查询算法来加速向量数据的存储和检索过程。

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

In [1]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

In [2]:
# 使用前配置自己的 api 到环境变量中如
import os
import openai
import zhipuai
import sys
sys.path.append('../..')
openai.api_base=r"https://api.qytech.top/v1"

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env fileopenai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key  = os.environ['OPENAI_API_KEY']
zhipuai.api_key = os.environ['ZHIPUAI_API_KEY']

os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

In [3]:
# 加载 PDF
loaders_chinese = [
    #PyMuPDFLoader("../../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf") # 南瓜书
    PyMuPDFLoader("/home/ysp/Downloads/test2/s71200.pdf")
    
    # 大家可以自行加入其他文件
]
docs = []
for loader in loaders_chinese:
    docs.extend(loader.load())
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)


# 定义 Embeddings
#embedding = OpenAIEmbeddings() 
embedding = HuggingFaceEmbeddings(model_name='moka-ai/m3e-base')
#embedding = ZhipuAIEmbeddings()

/home/ysp/softwares/anaconda/envs/llmenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
persist_directory = '../../data_base/vector_db/chroma'

In [20]:
!rm -rf '../../data_base/vector_db/chroma'  # 删除旧的数据库文件（如果文件夹中有文件的话），window电脑请手动删除

## 二、构建 Chroma 向量库

In [5]:
vectordb = Chroma.from_documents(
    documents=split_docs[:3000], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

在此之后，我们要确保通过运行 vectordb.persist 来持久化向量数据库，以便我们在未来的课程中使用。

让我们保存它，以便以后使用！

In [6]:
vectordb.persist()

大家也可以直接载入已经构建好的向量库

In [7]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [8]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：3000


## 三、通过向量数据库检索

### 3.1 相似度检索

In [9]:
question="目录的主要内容是什么"

In [10]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [11]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

检索到的第0个内容: 
目录
S7-1200 可编程控制器
28
系统手册, V4.6 11/2022, A5E02486685-AP
--------------
检索到的第1个内容: 
•
https://ww.xx.yy.zz/module.html - 有关本地机架中的模块和固件更新能力 (页 881)的信
息
•
https://ww.xx.yy.zz/communication.html - 有关网络地址、通信接口的物理属性和通信统
计的通信 (页 885)信息
•
https://ww.xx.yy.zz/variable.html - CPU 变量（变量）和 I/O (
--------------
检索到的第2个内容: 
久 CPU 内存。由于存在文件系统管理和相关的值，CPU 中所需的内存大于文件的大小。数
据日志的持久性内存保持分配状态，直至 CPU 以下列方式之一取消分配内存：
•
用户程序调用 DataLogDelete 指令
•
Web 服务器用户从 Web 服务器中删除数据日志
•
SIMATIC Automation Tool 用户从 SIMATIC Automation Tool 中删除数据日志
通
--------------


### 3.2 MMR 检索

如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。


核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [58]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [59]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
轿车的养护和保养
– 轿车外部养护和清洁  . . . . . . . . .
164
– 轿车内部养护和清洁  . . . . . . . . .
171
– 车轮和轮胎 . . . . . . . . . . . . . .
174
– 附件、改装、维修和零部件更换  . . .
184
– 用户信息  . . . . . . . . . . . . . . .
189
– 发动机故障与排气系
--------------
MMR 检索到的第1个内容: 
后视镜
 主题引言
该章节包含有关下列主题的信息：
车内后视镜  . . . . . . . . . . . . . . .
85
车外后视镜  . . . . . . . . . . . . . . .
86
补充信息和警告说明：
●
换档 ⇒ 第 110 页
●
制动、停车和驻车 ⇒ 第 116 页
警告
自动防眩后视镜含有电解液，镜面玻璃破碎时可
能会溢出。电解液对皮肤、眼睛和呼吸器官有刺
--------------
MMR 检索到的第2个内容: 
目录
使用说明书编写说明 . . . . . . . .
4
轿车概貌  . . . . . . . . . . . . . . .
5
驾驶室
– 驾驶员侧车门概貌（类型 A）  . . . . .
5
– 驾驶员侧车门概貌（类型 B）  . . . . .
6
– 驾驶员侧车内概貌  . . . . . . . . . .
7
– 副仪表板概貌
 . . . . . . . . . . . .
--------------


可以看到内容有了更多的差异。

## 四、构造检索式问答连

我们已经可以通过向量数据库找到最相关的内容了，接下来我们可以让 LLM 来用这些相关的内容回答我们的问题。

### 4.1 直接询问 LLM

基于 LangChain，我们可以构造一个使用 LLM 进行问答的检索式问答链，这是一种通过检索步骤进行问答的方法。我们可以通过传入一个语言模型和一个向量数据库来创建它作为检索器。然后，我们可以用问题作为查询调用它，得到一个答案。

In [60]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModel

In [64]:
llm = OpenAI(temperature=0)

In [67]:
# 可以使用 HuggingFacePipeline 本地搭建大语言模型
model_id = 'THUDM/chatglm2-6b-int4' # 采用 int 量化后的模型可以节省硬盘占用以及实时量化所需的运算资源
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True)
model = AutoModel.from_pretrained(model_id, trust_remote_code=True).half().quantize(4).cuda()
model = model.eval()
pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

llm = HuggingFacePipeline(pipeline=pipe)

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /THUDM/chatglm2-6b-int4/resolve/main/tokenizer_config.json (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1d6c6fd730>: Failed to establish a new connection: [Errno 111] Connection refused')))"), '(Request ID: 06fcf539-36cd-4731-9ab7-f3d0561d0c2e)')' thrown while requesting HEAD https://huggingface.co/THUDM/chatglm2-6b-int4/resolve/main/tokenizer_config.json


ProxyError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /THUDM/chatglm2-6b-int4/resolve/main/tokenizer_config.json (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f1d6c6fd730>: Failed to establish a new connection: [Errno 111] Connection refused')))"), '(Request ID: 06fcf539-36cd-4731-9ab7-f3d0561d0c2e)')

In [61]:
llm = ZhipuAILLM(model="chatglm_std", temperature=0)

In [65]:
# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [66]:
# 可以以该方式进行检索问答
question = "目录主要包含什么内容"
result = qa_chain({"query": question})
print(f"大语言模型的回答为：{result['result']}")

KeyboardInterrupt: 

### 4.2 结合 prompt 提问

对于 LLM 来说，prompt 可以让更好的发挥大模型的能力。


我们首先定义了一个提示模板。它包含一些关于如何使用下面的上下文片段的说明，然后有一个上下文变量的占位符。

In [16]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [17]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [20]:
question = " 2025 年大语言模型效果最好的是哪个模型"

In [21]:
result = qa_chain({"query": question})
print(f"LLM 对问题的回答：{result['result']}")

LLM 对问题的回答：2025年，大语言模型的效果将取决于各种技术的发展情况。目前，OpenAI 的 GPT-3 模型是最先进的大语言模型，它的效果非常出色。但是，随着技术的发展，2025 年可能会有更先进的模型出现，效果更好。感谢您的提问！


这里因为没有对应的信息，所以大语言模型只能回答不知道。您可以将知识库的内容调整为大语言模型综述的内容重新进行尝试。

In [22]:
print(f"向量数据库检索到的最相关的文档：{result['source_documents'][0]}")

向量数据库检索到的最相关的文档：page_content='在本模块，我们将与读者分享提升大语言模型应用效果的各种技巧和最佳实践。书中内容涵盖广泛，包括软件开发提示词设计、文本总结、推理、转换、扩展以及构建聊天机器人等语言模型典型应用场景。我们衷心希望该课程能激发读者的想象力，开发出更出色的语言模型应用。\n\n随着 LLM 的发展，其大致可以分为两种类型，后续称为基础 LLM 和指令微调（Instruction Tuned）LLM。基础LLM是基于文本训练数据，训练出预测下一个单词能力的模型。其通常通过在互联网和其他来源的大量数据上训练，来确定紧接着出现的最可能的词。例如，如果你以“从前，有一只独角兽”作为 Prompt ，基础 LLM 可能会继续预测“她与独角兽朋友共同生活在一片神奇森林中”。但是，如果你以“法国的首都是什么”为 Prompt ，则基础 LLM 可能会根据互联网上的文章，将回答预测为“法国最大的城市是什么？法国的人口是多少？”，因为互联网上的文章很可能是有关法国国家的问答题目列表。' metadata={'source': '../knowledge_base/prompt_engineering/1. 简介 Introduction.md'}


这种方法非常好，因为它只涉及对语言模型的一次调用。然而，它也有局限性，即如果文档太多，可能无法将它们全部适配到上下文窗口中。

langchain 提供了几种不同的处理文档的方法：

|     类型      |                                定义/区别                                |                              优点                              |                              缺点                              |
|-------------|---------------------------------------------------------------------|----------------------------------------------------------------|----------------------------------------------------------------|
|   Stuff     | 将整个文本内容一次性输入给大模型进行处理。                               | - 只调用大模型一次，节省计算资源和时间。<br>- 上下文信息完整，有助于理解整体语义。<br>- 适用于处理较短的文本内容。 | - 不适用于处理较长的文本内容，可能导致模型过载。                |
|   Refine    | 通过多次调用大模型逐步改进文本质量，进行多次迭代优化。                          | - 可以在多次迭代中逐步改进文本质量。<br>- 适用于需要进行多次迭代优化的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的文本质量。<br>- 不适用于实时性要求较高的场景。 |
| Map reduce  | 将大模型应用于每个文档，并将输出作为新文档传递给另一个模型，最终得到单个输出。               | - 可以对多个文档进行并行处理，提高处理效率。<br>- 可以通过多次迭代处理实现优化。<br>- 适用于需要对多个文档进行处理和合并的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的结果。<br>- 不适用于处理单个文档的场景。 |
| Map re-rank | 在每个文档上运行初始提示，为答案给出一个分数，返回得分最高的响应。                        | - 可以根据置信度对文档进行排序和选择，提高结果的准确性。<br>- 可以提供更可靠的答案。<br>- 适用于需要根据置信度对文档进行排序和选择的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要对多个文档进行评分和排序。<br>- 不适用于不需要对文档进行排序和选择的场景。 |

我们可以根据需要配置 chain_type 的参数，选择对应的处理方式。如：
```
RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
```